In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten, Activation, Dropout
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from keras.applications import MobileNetV2
import tensorflow.keras.optimizers as optimizers
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras import callbacks
import os.path
import re
from PIL import Image
import cv2
from keras.preprocessing.image import ImageDataGenerator

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
# Part 2 - Fitting the CNN to the images
datagen = ImageDataGenerator(
    rotation_range=20,  # Góc xoay từ -20 đến +20 độ
    width_shift_range=0.2,  # Dịch chuyển theo chiều rộng từ -20% đến +20%
    height_shift_range=0.2,  # Dịch chuyển theo chiều cao từ -20% đến +20%
    shear_range=0.2,  # Góc cắt từ -20 đến +20 độ
    zoom_range=0.2,  # Tỉ lệ thu phóng từ 80% đến 120%
    horizontal_flip=True,  # Lật ảnh theo chiều ngang ngẫu nhiên
    fill_mode='nearest'  # Sử dụng phương pháp gần nhất để điền các điểm ảnh sau biến đổi
)

# Áp dụng augmentation dữ liệu cho tập huấn luyện
train_generator = datagen.flow_from_directory(
    '/kaggle/input/baquamaster/New folder/data1/train',  # Đường dẫn tới thư mục chứa tập huấn luyện
    target_size=(150, 150),  # Kích thước đầu vào mà mô hình mong đợi
    batch_size=32,  # Kích thước batch
    class_mode='categorical'  # Chế độ phân loại nhiều lớp
)

val_generator = datagen.flow_from_directory(
    '/kaggle/input/baquamaster/New folder/data1/val',  # Đường dẫn tới thư mục chứa tập huấn luyện
    target_size=(150, 150),  # Kích thước đầu vào mà mô hình mong đợi
    batch_size=32,  # Kích thước batch
    class_mode='categorical'  # Chế độ phân loại nhiều lớp
)

Found 4224 images belonging to 2 classes.
Found 1056 images belonging to 2 classes.


In [4]:
Efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Create a new model
model = Sequential()

model.add(Efficientnet_model)

model.add(Flatten())

# Add dense layers for classification
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu')) 
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

16705208/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 5, 5, 1280)       4049571   
                                                                 
 flatten (Flatten)           (None, 32000)             0         
                                                                 
 dense (Dense)               (None, 1024)              32769024  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                         

In [5]:
optimizer = Adam(learning_rate=0.0001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])

In [6]:
model.fit(train_generator, epochs=50, validation_data=val_generator)

Epoch 1/50


2023-06-27 16:53:12.382283: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


132/132 [==============================] - 149s 688ms/step - loss: 1.1922 - acc: 0.5679 - val_loss: 0.6542 - val_acc: 0.7055
Epoch 2/50
132/132 [==============================] - 55s 418ms/step - loss: 0.8422 - acc: 0.5812 - val_loss: 0.6343 - val_acc: 0.7538
Epoch 3/50
132/132 [==============================] - 54s 410ms/step - loss: 0.6922 - acc: 0.6250 - val_loss: 0.5844 - val_acc: 0.8068
Epoch 4/50
132/132 [==============================] - 57s 429ms/step - loss: 0.5917 - acc: 0.6745 - val_loss: 0.4829 - val_acc: 0.8059
Epoch 5/50
132/132 [==============================] - 54s 409ms/step - loss: 0.5213 - acc: 0.7382 - val_loss: 0.4060 - val_acc: 0.8352
Epoch 6/50
132/132 [==============================] - 55s 418ms/step - loss: 0.4416 - acc: 0.7865 - val_loss: 0.3475 - val_acc: 0.8447
Epoch 7/50
132/132 [==============================] - 57s 429ms/step - loss: 0.4022 - acc: 0.8170 - val_loss: 0.3257 - val_acc: 0.8475
Epoch 8/50
132/132 [==============================] - 55s 415ms/s

In [7]:
# Evaluate the model
loss, accuracy = model.evaluate(val_generator)

print('Loss:', loss)
print("Accuracy: %.2f%%" % (accuracy * 100))

33/33 [==============================] - 10s 287ms/step - loss: 0.3482 - acc: 0.8902
Loss: 0.34822678565979004
Accuracy: 89.02%


In [8]:
IMAGE_SIZE = (150, 150)
DIR_test = '/kaggle/input/baquamaster/New folder/testset_images'
df_test = {
    'id' : [],
    'image' : []
}
for i in os.listdir(DIR_test):
    if '._' not in i:
        img_dir = DIR_test + '/' + i
        img_id = int(re.sub(r'\.(jpg|png|gif)$', '', i))
        image = Image.open(img_dir)
        image = image.resize(IMAGE_SIZE)
        image = np.array(image).reshape(150, 150, 3)
        df_test['image'].append(image)
        df_test['id'].append(img_id)

test_img_df = pd.DataFrame(df_test)

test_x = df_test['image']
test_x = np.asarray(test_x, dtype = np.float32)
test_x.shape

(1320, 150, 150, 3)

In [9]:
predictions = model.predict(test_x)     # Vector of probabilities
pred_labels = np.argmax(predictions, axis = 1) # We take the highest probability
pred_labels.shape

42/42 [==============================] - 3s 33ms/step


(1320,)

In [10]:
predictions[:,0]

array([9.9999893e-01, 7.5299422e-21, 4.8462251e-01, ..., 1.1870993e-22,
       7.1022916e-01, 1.5413305e-07], dtype=float32)

In [11]:
a = [123.68, 116.779, 103.939]
b = [58.393, 57.12, 57.375]
def load_data1():
    datasets = ['/kaggle/input/baquamaster/New folder/testset_images']
    output = []

    IMAGE_SIZE = (150, 150)  # Kích thước ảnh

    # Iterate through training and test sets
    for dataset in datasets:
    
        file_name = []
        images = []

        print("Loading {}".format(dataset))

        # Iterate through each folder corresponding to a category

        # Iterate through each image in our folder
        for file in os.listdir(dataset):
            # Get the path name of the image
            img_path = os.path.join(dataset, file)
#             print(img_path)
            image = cv2.imread(img_path)
            image = (image - a)/b
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)
            if file.startswith('._'):
                continue
            file_name.append(file.split('.')[0])

            # Open and resize the img
#             image = cv2.imread(img_path)
#             image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#             image = cv2.resize(image, IMAGE_SIZE)

            # Append the image and its corresponding label to the output
            images.append(image)

        images = np.array(images, dtype='float32')
        output.append(images)
        output.append(file_name)

    return output


In [12]:
test = load_data1()

Loading /kaggle/input/baquamaster/New folder/testset_images


In [13]:
result = []
columns = ['id', 'label']
result.append(columns)
for i in range(len(test_x)):
    row = [test[1][i],np.argmax(predictions[i])]
#     print(predictions[i][0])
    result.append(row)
import csv
for i in range(len(result)):
    if result[i][1] == 0:
        result[i][1] = 1
    elif result[i][1] == 'label':
        pass
    else:
        result[i][1] = 0

In [14]:
file_path = '/kaggle/working/Result1.csv'

with open(file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(result)

print("CSV file exported successfully.")

CSV file exported successfully.
